In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import geopandas as gpd
import os
import fiona
import rasterio
import rasterio.mask
import imageio
import itertools
import shapely
from shapely.wkt import loads

# Pop

In [28]:
pop = gpd.read_file(r"D:\bike\code\metro\age_buffer\pop1500.shp", encoding='utf-8')
pop = pop.rename(columns = {'SAMPLE_1':'Pop'})

In [29]:
pop

,stationnam,Pop,geometry
0,罗湖,NaN,POINT (820809.311 2495070.993)
1,人民南,NaN,POINT (820748.669 2495450.082)
2,国贸,258.015625,POINT (820816.745 2495913.339)
3,老街,328.348114,POINT (820603.435 2496413.511)
4,大剧院,324.088257,POINT (819669.497 2496124.436)
...,...,...,...
229,福永,161.854813,POINT (788339.380 2510107.183)
230,机场,NaN,POINT (789223.295 2504633.877)
231,碧海湾,214.016708,POINT (793691.303 2499293.417)
232,宝安,205.913666,POINT (796235.679 2497072.139)


In [30]:
pop = pop.fillna(0)

In [31]:
rail_station_count =pd.read_csv(r'D:\bike\code\metro\Preprocessing-bike data\metro_area.csv').drop(columns = 'Unnamed: 0')

geometry = rail_station_count['geometry'].map(shapely.wkt.loads)
rail_station_count = rail_station_count.drop('geometry', axis=1)
rail_station_count = gpd.GeoDataFrame(rail_station_count, geometry=geometry)
rail_station_count = rail_station_count.set_crs("EPSG:4326")

# buffer = 300
rail_station = rail_station_count.to_crs("EPSG:32649")
rail_station['buffer'] = rail_station.buffer(1500)

rail_station_buffer = gpd.GeoDataFrame(rail_station,geometry = 'buffer')
rail_station_buffer = rail_station_buffer.drop(columns = 'index_right')

In [32]:
rail_station_buffer = rail_station_buffer.merge(pop,on = 'stationnam')


In [33]:
rail_station_buffer['area'] = rail_station_buffer.area

In [34]:
rail_station_buffer

,linename,stationnam,sl,poiid,lg,sp,lon,lat,hex_id,cluster_la,functional,function_1,geometry_x,buffer,Pop,geometry_y,area
0,地铁1号线(罗宝线),罗湖,"114.118666,22.532083",BV10249978,2,luo hu,114.118666,22.532083,30444,0,Rural,4,POINT (820809.311 2495070.993),"POLYGON ((822309.311 2495070.993, 822302.088 2...",0.000000,POINT (820809.311 2495070.993),7.057234e+06
1,地铁9号线,人民南,"114.118154,22.535514",BV10447615,0,ren min nan,114.118154,22.535514,30444,0,Rural,4,POINT (820748.669 2495450.082),"POLYGON ((822248.669 2495450.082, 822241.446 2...",0.000000,POINT (820748.669 2495450.082),7.057234e+06
2,地铁1号线(罗宝线),国贸,"114.118909,22.539680",BV10243546,2,guo mao,114.118909,22.539680,30445,1,Center,1,POINT (820816.745 2495913.339),"POLYGON ((822316.745 2495913.339, 822309.522 2...",258.015625,POINT (820816.745 2495913.339),7.057234e+06
3,地铁1号线(罗宝线),老街,"114.116939,22.544232",BV10249964,2,lao jie,114.116939,22.544232,30441,1,Center,1,POINT (820603.435 2496413.511),"POLYGON ((822103.435 2496413.511, 822096.212 2...",328.348114,POINT (820603.435 2496413.511),7.057234e+06
4,地铁1号线(罗宝线),大剧院,"114.107811,22.541800",BV10246013,3,DaJuYuan,114.107811,22.541800,30440,1,Center,1,POINT (819669.497 2496124.436),"POLYGON ((821169.497 2496124.436, 821162.274 2...",324.088257,POINT (819669.497 2496124.436),7.057234e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,地铁11号线,福永,"113.806076,22.673562",BV10249452,0,FuYong,113.806076,22.673562,18097,2,Sub-center,2,POINT (788339.380 2510107.183),"POLYGON ((789839.380 2510107.183, 789832.157 2...",161.854813,POINT (788339.380 2510107.183),7.057234e+06
230,地铁11号线,机场,"113.813663,22.624026",BV10527864,6,ji chang,113.813663,22.624026,10976,3,Outskirt,3,POINT (789223.295 2504633.877),"POLYGON ((790723.295 2504633.877, 790716.072 2...",0.000000,POINT (789223.295 2504633.877),7.057234e+06
231,地铁11号线,碧海湾,"113.856093,22.575073",BV10527859,6,bi hai wan,113.856093,22.575073,24808,1,Center,1,POINT (793691.303 2499293.417),"POLYGON ((795191.303 2499293.417, 795184.080 2...",214.016708,POINT (793691.303 2499293.417),7.057234e+06
232,地铁11号线,宝安,"113.880396,22.554590",BV10527865,6,bao an,113.880396,22.554590,33438,1,Center,1,POINT (796235.679 2497072.139),"POLYGON ((797735.679 2497072.139, 797728.456 2...",205.913666,POINT (796235.679 2497072.139),7.057234e+06


In [35]:
rail_station_buffer['pop_density'] = rail_station_buffer['Pop']/(rail_station_buffer['area'])*1000000*100/1000

In [36]:
Pop = rail_station_buffer[['stationnam','Pop','pop_density','area']]

In [37]:
Pop.describe()

,Pop,pop_density,area
count,234.000000,234.000000,2.340000e+02
mean,160.896257,2.279877,7.057234e+06
std,103.244559,1.462961,2.426629e-08
min,0.000000,0.000000,7.057234e+06
25%,88.922112,1.260014,7.057234e+06
50%,160.876770,2.279601,7.057234e+06
75%,244.646393,3.466604,7.057234e+06
max,331.161804,4.692516,7.057234e+06


In [38]:
Pop.to_csv(r"D:\bike\code\metro\Preprocessing-bike data\code\day_buffer\variable\Pop_density_variables_1500.csv",encoding = 'utf-8-sig')

# Age

In [ ]:
hex_area = gpd.read_file(r"D:\bike\code\metro\Preprocessing-bike data\hex_shp\Differ_Area.shp")
hex_area

In [ ]:
hex_area = hex_area.set_crs('epsg:4326')
hex_area_copy = hex_area.to_crs('epsg:32649')
 # hex_area_copy.to_file(r'D:\bike\code\metro\Preprocessing-bike data\hex_shp\Differ_Area_copy.shp')

In [ ]:
hex_area_copy['area']= hex_area_copy.area

In [ ]:
age = gpd.read_file(r"D:\bike\code\metro\age\age\chn_f_0_2020.shp")
age = age.fillna(0)

In [ ]:
age = age.rename(columns = {'SAMPLE_1':'chn_f_0_2020'})

In [ ]:
age.head()

In [ ]:
age_copy = age[['hex_id','chn_f_0_2020']]
hex_area_copy = hex_area_copy.merge(age_copy,on = 'hex_id')


In [ ]:
hex_area_copy

In [ ]:
hex_area_copy['chn_f_0_2020'].sum() #深圳人口大概9.8万新生儿 #单位为100人

In [ ]:
hex_area_copy['chn_f_0_2020'] = hex_area_copy['chn_f_0_2020']*100

In [ ]:
hex_area_copy.describe()

In [ ]:
age = gpd.GeoDataFrame(hex_area_copy,geometry ='geometry')
age = age.set_crs('epsg:32649')
age = age.to_crs('epsg:4326')
# age.to_file(r"D:\bike\code\metro\Preprocessing-bike data\hex_shp\age.shp")

In [ ]:
region = gpd.read_file(r"D:\bike\shp\shenzhen\shenzhen.shp")

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))
age.plot(column='chn_f_0_2020',cmap='Reds',ax=ax,legend=True,
            legend_kwds={'orientation':'vertical','shrink':0.45,'label':'chn_f_0_2020 (persons/km2)'})
region.plot(ax=ax,facecolor='none',edgecolor='k')

# Age

In [ ]:
import os
folder = r"D:\bike\code\metro\age\age"
shp_list = []
for i in os.walk(folder):
    for m in i[2]:
        if m.endswith('shp'):
            shp_list.append(m)

In [ ]:
shp_list

In [ ]:
hex_area = gpd.read_file(r"D:\bike\code\metro\Preprocessing-bike data\hex_shp\Differ_Area.shp")
hex_area
hex_area = hex_area.set_crs('epsg:4326')
hex_area_copy = hex_area.to_crs('epsg:32649')
hex_area_copy['area']= hex_area_copy.area

for x in shp_list:
    
    age = gpd.read_file(r"D:\bike\code\metro\age\age\{}".format(x))
    age = age.fillna(0)
    age['SAMPLE_1'] = age['SAMPLE_1']*100
    age = age.rename(columns = {'SAMPLE_1':'{}'.format(x)})
    age = age[['hex_id','{}'.format(x)]]
    hex_area_copy = hex_area_copy.merge(age,on = 'hex_id')
    

In [ ]:
hex_area_copy

In [ ]:
age = hex_area_copy 

In [ ]:
age['female'] = age[[col for col in age.columns if "chn_f" in col]].sum(axis=1)
age['male'] = age[[col for col in age.columns if "chn_m" in col]].sum(axis=1)
age['male%'] = age['male']/(age['female']+age['male'])
age['female%'] = age['female']/(age['female']+age['male'])

In [ ]:
age['age_0'] = age[[col for col in age.columns if "_0_2020" in col]].sum(axis=1)
age['age_1'] = age[[col for col in age.columns if "_1_2020" in col]].sum(axis=1)
age['age_5'] = age[[col for col in age.columns if "_5_2020" in col]].sum(axis=1)
age['age_10'] = age[[col for col in age.columns if "_10_2020" in col]].sum(axis=1)
age['age_15'] = age[[col for col in age.columns if "_15_2020" in col]].sum(axis=1)
age['age_20'] = age[[col for col in age.columns if "_20_2020" in col]].sum(axis=1)
age['age_25'] = age[[col for col in age.columns if "_25_2020" in col]].sum(axis=1)
age['age_30'] = age[[col for col in age.columns if "_30_2020" in col]].sum(axis=1)
age['age_35'] = age[[col for col in age.columns if "_35_2020" in col]].sum(axis=1)
age['age_40'] = age[[col for col in age.columns if "_40_2020" in col]].sum(axis=1)
age['age_45'] = age[[col for col in age.columns if "_45_2020" in col]].sum(axis=1)
age['age_50'] = age[[col for col in age.columns if "_50_2020" in col]].sum(axis=1)
age['age_55'] = age[[col for col in age.columns if "_55_2020" in col]].sum(axis=1)
age['age_60'] = age[[col for col in age.columns if "_60_2020" in col]].sum(axis=1)
age['age_65'] = age[[col for col in age.columns if "_65_2020" in col]].sum(axis=1)
age['age_70'] = age[[col for col in age.columns if "_70_2020" in col]].sum(axis=1)
age['age_75'] = age[[col for col in age.columns if "_75_2020" in col]].sum(axis=1)
age['age_80'] = age[[col for col in age.columns if "_80_2020" in col]].sum(axis=1)

In [ ]:
age['all'] = age[[col for col in age.columns if "age_" in col]].sum(axis=1)
age['0-14'] = age['age_0'] + age['age_1'] + age['age_5'] + age['age_10']
age['60+'] = age['age_60'] + age['age_65'] + age['age_70'] + age['age_75']+ age['age_80']
age['15-60'] = age['all'] - age['0-14'] - age['60+']
age['0-14%'] = age['0-14']/age['all']
age['15-60%'] = age['15-60']/age['all']
age['60+%'] = age['60+']/age['all']

In [ ]:
age.to_csv(r"D:\bike\code\metro\Preprocessing-bike data\age.csv")

In [ ]:
age_variable = age[['hex_id', 'cluster_la', 'functional', 'function_1', 'geometry',
       'area','0-14','15-60','60+','all','0-14%','15-60%','60+%','female','male','male%','female%']]

In [ ]:
age_variable.to_csv(r"D:\bike\code\metro\Preprocessing-bike data\age_variable.csv")